In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:

def scrape_amazon(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    
    driver = webdriver.Chrome(options=options)
    
    scraped_data = []

    try:
        print("Fetching the Amazon webpage...")
        driver.get(url)

        print("Waiting for search results to load...")
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-component-type='s-search-result']")))
        
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', {'data-component-type': 's-search-result'})

        if not products:
            raise ValueError("No product containers found. Amazon might be blocking the request.")

        print(f"Found {len(products)} product containers. Extracting data...")

        for i, product in enumerate(products):
        
            title_element = product.find('h2')
            title = title_element.text.strip() if title_element else 'N/A'

        
            price_element = product.find('span', class_='a-offscreen')
            price = price_element.text.strip() if price_element else 'N/A'
            
        
            rating_element = product.find('span', class_='a-icon-alt')
            rating = rating_element.text.strip() if rating_element else 'N/A'
            
        
            print(f"  -> Processing item {i+1}: Title='{title}', Price='{price}'")
            
            if price != 'N/A' and title != 'N/A':
                scraped_data.append({
                    'Title': title,
                    'Price': price,
                    'Rating': rating
                })
        
        return pd.DataFrame(scraped_data)

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Saving debug files...")
        driver.save_screenshot('amazon_debug_screenshot.png')
        with open('amazon_debug_page.html', 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
        print("Saved amazon_debug_screenshot.png and amazon_debug_page.html.")
        return None
    finally:
        driver.quit()
        print("Scraping complete.")


if __name__ == "__main__":
    amazon_url = "https://www.amazon.in/s?k=car+cover"
    results_df = scrape_amazon(amazon_url)

    if results_df is not None and not results_df.empty:
        print("\n--- Amazon Search Results ---")
        print(results_df.to_string())
    else:
        print("\nNo results were successfully scraped.")

Fetching the Amazon webpage...
Waiting for search results to load...
Found 60 product containers. Extracting data...
  -> Processing item 1: Title='NEODRIFT 'NeoTech' Car Cover for Skoda Slavia (100% Water-Resistant, All Weather Protection, Tailored Fit, Multi-Layered & Breathable Fabric) (Colour: Blue)', Price='₹4,144'
  -> Processing item 2: Title='PROTEGO All Weather Protection N-Series Car Cover for Maruti Ciaz | Anti Dust, 100% Waterproof, Custom Fit, UV Proof - Multi-Layered, Elastic Grip, Buckle Hook with Mirror Pocket | (Blue)', Price='₹2,566'
  -> Processing item 3: Title='PROTEGO 4x4 Heavy Duty Fabric All Weather Car Cover for Safari Old | Dog Proof, 100% Waterproof, Custom Fit, Elastic Grip, 4+ Years Life, Durable, Mirror Pocket | (Army Military)', Price='₹3,149'
  -> Processing item 4: Title='PROTEGO All Weather Protection Car Cover for Hyundai i10 Grand Nios | Anti Dust, 100% Waterproof, Durable - Triple Stitched Elastic Grip with Mirror Pocket|(Military Jungle Print, Camo